In [29]:
import numpy as np
from dataclasses import dataclass
from random import choice, random, randint
from functools import reduce
from copy import copy

In [30]:
PROBLEM_SIZE = 100
NUM_SETS = 50
# In this problem the state is a array of boolean that tells me if the SETS[i] is taken or not
SETS = tuple(np.array([random() < .3 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))

POPULATION_SIZE = 30 # in an Evolutionary startegy, this would have been μ
OFFSPRING_SIZE = 20 # in an Evolutionary strategy, this whould have been λ (but it should be much largen than μ)
# Since OFFSPRING_SIZE < POPULATION_SIZE ---> Steady-State algorithm (in GA terminology). 
# We are creating 20 new individuals every generation and putting them back in the population (50 individuals total) 
# and then we are killing the worst 20 (to come back to 30).
# In evolutionary strategy (ES), this would have been a + strategy: we add the individual to the population and then we do 
# the selection
TOURNAMENT_SIZE = 2
MUTATION_PROBABILITY = .2 # Probability an individual get mutated passing from a generation to the next one

In [31]:
def fitness(state):
    cost = sum(state) # How many sets I am using
    covered_elements = np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)])
        )
    )
    return covered_elements, -cost

In [32]:
# The genotype is chosen with the genetic operators because you have to choose the encoding and the way this can be mutated
# and recombined in order to pass from a generation to the next one.

@dataclass
class Individual:
    genotype: list[bool]
    fitness: tuple

# This is the function to select the best parent in my population
def select_parent(population):
    # I'm picking TOURNAMENT SIZE random individuals and then I pick the best one out of it: TOURNAMENT STRATEGY
    candidates = [choice(population) for _ in range(TOURNAMENT_SIZE)]
    champion = max(candidates, key = lambda i: i.fitness)
    return champion

# Function to mutate an individual
def mutate(ind: Individual):
    mutated = copy(ind)
    index = randint(0, NUM_SETS - 1) 
    mutated.genotype[index] = not mutated.genotype[index]
    # Dopo che abbiamo modificato l'individuo la fitness associatagli deve essere resettata
    mutated.fitness = None
    return mutated

# Possible function to make the xover
def one_cut_xover(ind1: Individual, ind2: Individual):
    cut_point = randint(0, NUM_SETS-1)
    new_ind = Individual(genotype = ind1.genotype[:cut_point] + ind2.genotype[cut_point:],
                         fitness = None)
    assert len(new_ind.genotype) == NUM_SETS
    return new_ind

In [33]:
# The population is a set of random individuals
population = [Individual(genotype=[choice((True, False)) for _ in range(NUM_SETS)], fitness = None) 
              for _ in range(POPULATION_SIZE)]

for i in population:
    i.fitness = fitness(i.genotype)

In [34]:
# In each step (generation) the first thing to do is to pick the PARENTS

for generation in range(10):
    # The goal is to create an offspring of OFFSPRING_SIZE new individuals mutated or recombined from the PARENTS
    offspring = list()
    for _ in range(OFFSPRING_SIZE):
        # In GA Algorithms we have 2 choice as genetic operators startegies:
        # 1) selecting between crossover (recombination) or mutation
        # 2) you do recombination and then, with a certain probability, you mutate the offspring
        # We are using (1)
        if random() < MUTATION_PROBABILITY:
            # mutation
            p = select_parent(population)
            o = mutate(p)
        else:
            # xover
            p1 = select_parent(population)
            p2 = select_parent(population)
            o = one_cut_xover(p1, p2)
        offspring.append(o)
    # just out of this for we have an offspring for the new generation

    # Now I evaluate the new offspring
    for o in offspring:
        o.fitness = fitness(o.genotype)
    # I add the offspring to the population (since it is a steady state algorithm)
    population.extend(offspring)
    # Then I sort comparing the fitnesses
    population.sort(key = lambda i: i.fitness, reverse = True)
    # SURVIVAL SELECTION: I pick the POPULATION_SIZE best individuals of the new population to pass to the next generation
    population = population[:POPULATION_SIZE]
    print(f'best individual of #{generation+1} generation has the following fitness: {population[0].fitness}')

best individual of #1 generation has the following fitness: (100, -18)
best individual of #2 generation has the following fitness: (100, -17)
best individual of #3 generation has the following fitness: (100, -14)
best individual of #4 generation has the following fitness: (100, -14)
best individual of #5 generation has the following fitness: (100, -13)
best individual of #6 generation has the following fitness: (100, -13)
best individual of #7 generation has the following fitness: (100, -12)
best individual of #8 generation has the following fitness: (100, -12)
best individual of #9 generation has the following fitness: (100, -12)
best individual of #10 generation has the following fitness: (100, -12)
